In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import LabelEncoder

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("../datasets/NationalNames.csv")

In [4]:
vocab = dict(zip(string.ascii_lowercase, range(1, 27)))

In [5]:
df["Name"] = df["Name"].apply(str.lower)

In [6]:
def convert(item):
    return [vocab[ch] for ch in item if ch in vocab]

In [7]:
df["Name"] = df["Name"].apply(convert)

In [8]:
X = np.array(df["Name"])
y = np.array(df["Gender"])

In [9]:
le = LabelEncoder()

In [10]:
le.fit(y)

LabelEncoder()

In [11]:
y = le.transform(y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
X_train_mod = pad_sequences(X_train, maxlen=15)
X_test_mod = pad_sequences(X_test, maxlen=15)

In [15]:
X_train_mod.shape

(1223040, 15)

In [16]:
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model

In [17]:
y_train[:10]

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [18]:
in_layer = Input(shape=(15,))
embedding = Embedding(input_dim=27, output_dim=4)(in_layer)
rnn_layer = LSTM(units=50)(embedding)
out_layer = Dense(1, activation="sigmoid")(rnn_layer)

In [19]:
model = Model(in_layer, out_layer)

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 4)             108       
_________________________________________________________________
lstm (LSTM)                  (None, 50)                11000     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 11,159
Trainable params: 11,159
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

In [25]:
model.fit(X_train_mod[10000:50000], y_train[10000:50000], epochs=20, batch_size=1000)

Epoch 1/20
40/40 [==============================] - 5s 134ms/step - loss: 0.1550 - accuracy: 0.7779
Epoch 2/20
40/40 [==============================] - 6s 156ms/step - loss: 0.1535 - accuracy: 0.7813
Epoch 3/20
40/40 [==============================] - 6s 158ms/step - loss: 0.1528 - accuracy: 0.7822
Epoch 4/20
40/40 [==============================] - 6s 160ms/step - loss: 0.1525 - accuracy: 0.7835
Epoch 5/20
40/40 [==============================] - 7s 167ms/step - loss: 0.1523 - accuracy: 0.7831
Epoch 6/20
40/40 [==============================] - 7s 182ms/step - loss: 0.1522 - accuracy: 0.7829
Epoch 7/20
40/40 [==============================] - 7s 172ms/step - loss: 0.1518 - accuracy: 0.7834
Epoch 8/20
40/40 [==============================] - 7s 164ms/step - loss: 0.1516 - accuracy: 0.7833
Epoch 9/20
40/40 [==============================] - 7s 173ms/step - loss: 0.1513 - accuracy: 0.7836
Epoch 10/20
40/40 [==============================] - 7s 174ms/step - loss: 0.1513 - accuracy: 0.7839

KeyboardInterrupt: 

In [27]:
model.evaluate(X_test_mod[:10000], y_test[:10000], verbose=0)

[0.15363582968711853, 0.7825999855995178]

In [69]:
name = "katreena"

In [70]:
X_work = pad_sequences([convert(name)], maxlen=15)

In [71]:
X_work

array([[ 0,  0,  0,  0,  0,  0,  0, 11,  1, 20, 18,  5,  5, 14,  1]],
      dtype=int32)

In [72]:
model.predict(X_work)

array([[0.01234114]], dtype=float32)

In [64]:
le.classes_

array(['F', 'M'], dtype=object)